In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt


In [ ]:
# Attempt to unzip zipped file, failed
#import gzip
#with gzip.open('fashion_mnist/train-images-idx3-ubyte.gz') as f:

learning_set = np.fromfile('fashion_mnist/train-images-idx3-ubyte', dtype='uint8')
learning_lab = np.fromfile('fashion_mnist/train-labels-idx1-ubyte', dtype='uint8')
test_set = np.fromfile('fashion_mnist/t10k-images-idx3-ubyte', dtype='uint8')
test_lab = np.fromfile('fashion_mnist/t10k-labels-idx1-ubyte', dtype='uint8')

In [ ]:
learning_set.shape

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']
print(learning_set[4:8])

Preprocessing MNIST data into separate folders

In [ ]:
# Importing PIL for image manipulation
import PIL

In [ ]:
plt.grid(False)
plt.xticks([])
plt.yticks([])  
img = np.reshape(learning_set[range(16,28*28+16)], (28,28))
plt.imshow(img, cmap=plt.cm.binary)

In [ ]:
np.reshape(learning_set[16:], (60000, 28, 28, 1)).shape

In [ ]:
# Loading data straight from folders, will work in 2.4.0

# fashion_mnist_train = tf.keras.preprocessing.image_dataset_from_directory(
#     train_path,
#     labels="inferred",import tensorflow as tf

#     label_mode="int",
#     class_names=['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
#                'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot'],
#     color_mode='grayscale',
#     batch_size=32,
#     image_size=(28, 28),
#     shuffle=True,
#     seed=None,
#     validation_split=None,
#     subset=None,
#     interpolation="bilinear",
#     follow_links=False,
# )

# fashion_mnist_test = tf.keras.preprocessing.image_dataset_from_directory(
#     test_path,
#     labels="inferred",
#     label_mode="int",
#     class_names=['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
#                'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot'],
#     color_mode='grayscale',
#     batch_size=32,
#     image_size=(28, 28),
#     shuffle=True,
#     seed=None,
#     validation_split=None,
#     subset=None,
#     interpolation="bilinear",
#     follow_links=False,
# )

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

train_examples = 60000
test_examples = 10000
train_dataset = tf.data.Dataset.from_tensor_slices(
    (np.reshape(learning_set[16:], (60000, 28, 28, 1)), # Input
     learning_lab[8:]) # Output
)
test_dataset  = tf.data.Dataset.from_tensor_slices(
    (np.reshape(test_set[16:], (10000, 28, 28, 1)), # Input
     test_lab[8:]) # Output
)


Testing whether dataset was loaded properly, should show a boot:

In [ ]:
plt.grid(False)
plt.xticks([])
plt.yticks([])  
img = np.reshape(train_dataset.as_numpy_iterator().next()[0], (28,28))
plt.imshow(img, cmap=plt.cm.binary)

Dataset loaded, learning time

In [ ]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)

train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()

In [ ]:
# #layerInput = tf.keras.layers.Input(shape=(28, 28, 1))
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(32, (3,3), activation=tf.nn.relu,
#                            input_shape=(28, 28, 1)),
#     tf.keras.layers.MaxPooling2D((2, 2), strides=2),
#     tf.keras.layers.Dropout(.1, input_shape=(28,28,32*64)),
#     tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
#     tf.keras.layers.MaxPooling2D((2, 2), strides=2),
#     tf.keras.layers.Dropout(.1, input_shape=(28,28,32*64)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation=tf.nn.relu),
#     tf.keras.layers.Dense(10, activation=tf.nn.softmax)
# ])

In [ ]:
tf.keras.backend.clear_session()
    
#layerInput = tf.keras.layers.Input(shape=(28, 28, 1))
Linput = tf.keras.Input(dtype = tf.float32, shape = (28, 28, 1), name = 'Target')
prevLayer = tf.keras.layers.Conv2D(32, (3,3), activation=tf.nn.relu, input_shape=(28, 28, 1))(Linput)
prevLayer = tf.keras.layers.MaxPooling2D((2, 2), strides=2)(prevLayer)
prevLayer = tf.keras.layers.Dropout(.1, input_shape=(28,28,32*64))(prevLayer)
prevLayer = tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu)(prevLayer)
prevLayer = tf.keras.layers.MaxPooling2D((2, 2), strides=2)(prevLayer)
prevLayer = tf.keras.layers.Dropout(.1, input_shape=(28,28,32*64))(prevLayer)
prevLayer = tf.keras.layers.Flatten()(prevLayer)
prevLayer = tf.keras.layers.Dense(128, activation=tf.nn.relu)(prevLayer)
prevLayer = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(prevLayer)

model = tf.keras.Model(inputs = Linput, outputs = prevLayer)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 32
train_dataset = train_dataset.cache().repeat().shuffle(train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

In [ ]:
model.summary()

In [ ]:
model.fit(train_dataset, epochs=3, steps_per_epoch=math.ceil(train_examples/BATCH_SIZE))

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

In [ ]:
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img[...,0], cmap=plt.cm.binary)
    

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [ ]:
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)

## Testing model saving/loading and progress tracking/profiling capabilities

In [ ]:
def reset_weights(model):
    for layer in model.layers:
        if isinstance(layer, tf.keras.Model): #if you're using a model as a layer
            reset_weights(layer) #apply function recursively
            continue

        #where are the initializers?
        if hasattr(layer, 'cell'):
            init_container = layer.cell
        else:
            init_container = layer

        for key, initializer in init_container.__dict__.items():
            if "initializer" not in key: #is this item an initializer?
                  continue #if no, skip it

            # find the corresponding variable, like the kernel or the bias
            if key == 'recurrent_initializer': #special case check
                var = getattr(init_container, 'recurrent_kernel')
            else:
                var = getattr(init_container, key.replace("_initializer", ""))

            var.assign(initializer(var.shape, var.dtype))
            #use the initializer

Tensorboard callback for saving progress of learning

In [ ]:
callbackTB = tf.keras.callbacks.TensorBoard(
    log_dir="logs",
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq="epoch",
    profile_batch=2,
    embeddings_freq=3,
    embeddings_metadata=None
)


Callback for saving values in csv format with CSVLogger

In [ ]:
callbackCSV = tf.keras.callbacks.CSVLogger("fashion-csvlog.csv", separator=",", append=False)

Callback for saving model when given conditions have been met. In this example when loss value reaches minimum

In [ ]:
callbackCheckpoint = tf.keras.callbacks.ModelCheckpoint(
    "fashion-model",
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    save_freq="epoch",
    options=None
)


In [ ]:
reset_weights(model)

In [ ]:
# model.fit(train_dataset, 
#           epochs=1, 
#           steps_per_epoch=math.ceil(train_examples/BATCH_SIZE), 
#           validation_data=test_dataset,
#           callbacks=[callbackCheckpoint,
#           callbackCSV,
#           callbackTB]
#          )

# Tensorboard logging gradient

In [ ]:
reset_weights(model)

In [ ]:
class ExtendedTensorBoard(tf.keras.callbacks.TensorBoard):
    def _log_gradients(self, epoch):
        writer = self._get_writer(self._train_run_name)

        with writer.as_default(), tf.GradientTape() as g:
            # here we use test data to calculate the gradients
## Wrong approach dont do that, use internal variables of actual data not external lists
            features, y_true = list(val_dataset.batch(100).take(1))[0]

            y_pred = self.model(features)  # forward-propagation
            loss = self.model.compiled_loss(y_true=y_true, y_pred=y_pred)  # calculate loss
            gradients = g.gradient(loss, self.model.trainable_weights)  # back-propagation

            # In eager mode, grads does not have name, so we get names from model.trainable_weights
            for weights, grads in zip(self.model.trainable_weights, gradients):
                tf.summary.histogram(
                    weights.name.replace(':', '_') + '_grads', data=grads, step=epoch)

        writer.flush()

    def on_epoch_end(self, epoch, logs=None):
        # This function overwrites the on_epoch_end in tf.keras.callbacks.TensorBoard
        # but we do need to run the original on_epoch_end, so here we use the super function.
        super(ExtendedTensorBoard, self).on_epoch_end(epoch, logs=logs)

        if self.histogram_freq and epoch % self.histogram_freq == 0:
            self._log_gradients(epoch)

In [ ]:
gradient_log = []

class GradientsLoggerTBCallback(tf.keras.callbacks.TensorBoard):
    def __init__(self, gradient_reference, *args, **kwargs):
        super(GradientsLoggerTBCallback, self).__init__(*args, **kwargs)
        self._gradient_ref = gradient_reference
        self._epoch = 1
        self.once = True
        
    def _get_gradient(self, model):
        with tf.GradientTape() as tape:
            y_pred = model(self._gradient_ref[0], training=True)  # Forward pass
            loss = self.model.compiled_loss(y_true=self._gradient_ref[1], y_pred=y_pred)

        # Compute gradients
        trainable_vars = model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        return gradients

    def _log_gradients(self, epoch):
        writer = self._get_writer(self._train_run_name)
        gradients = self._get_gradient(model)
        
        gradient_log.append(gradients)
        with writer.as_default():
            # Getting names from model.trainable_weights
            for weights, grads in zip(self.model.trainable_weights, gradients):
                tf.summary.histogram(
                    weights.name.replace(':', '_') + '_grads', data=grads, step=epoch)
        writer.flush()

    def on_epoch_end(self, epoch, logs=None):
        super(GradientsLoggerTBCallback, self).on_epoch_end(epoch, logs=logs)
        
        self._epoch += 1
        if self.histogram_freq and epoch % self.histogram_freq == 0:
            self._log_gradients(epoch)
    
    #def on_train_batch_end(self, batch, logs=None):
        #if self.histogram_freq and self._epoch % self.histogram_freq == 0 and self.once:
            #print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))
            #print(self.__dir__())
            #print(logs)
            #self.once = False
            


In [ ]:
callbackGradientsLogger = GradientsLoggerTBCallback(
    list(test_dataset.take(1))[0],
    log_dir="/qarr/studia/magister/models/logs",
    histogram_freq=1,
    write_graph=True,
    update_freq="epoch",
)

In [ ]:
model.fit(train_dataset, 
          epochs=2, 
          steps_per_epoch=math.ceil(train_examples/BATCH_SIZE), 
          validation_data=test_dataset,
          callbacks=[callbackGradientsLogger]
         )

In [ ]:
def describe(x):
    try:
        return f'{x.shape}'
    except AttributeError:
        return '[' + ', \n'.join([describe(q) for q in x]) + ']'


In [ ]:
print(describe(gradient_log))

In [ ]:
[w.name for w in model.trainable_weights]